# Parking_Demand_Predict

[주차수요 예측 AI 경진대회](https://dacon.io/competitions/official/235745/overview/description)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


pd.options.display.float_format = '{:.4f}'.format
data_path = "./data/"
train_file = f"{data_path}train_V2.csv"
test_file = f"{data_path}test_V2.csv"
sample_submit_file = f"{data_path}sample_submission_V2.csv"